In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
import os
import sys
import PIL
import PIL.Image
import tensorflow_datasets as tfds
import pathlib
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### COLLECTING DATA 

In [2]:
train_data_generator = ImageDataGenerator(rescale=1/255.0, validation_split=0.2)

data_dir = pathlib.Path('tanda_baca')
train_ds = train_data_generator.flow_from_directory(
  data_dir,
  subset="training",
  seed=123,
  target_size=(28, 28),
  class_mode="categorical",
  batch_size=32,
  color_mode="grayscale")

val_ds = train_data_generator.flow_from_directory(
  data_dir,
  subset="validation",
  seed=123,
  target_size=(28, 28),
  class_mode="categorical",
  batch_size=32,
  color_mode="grayscale")


Found 488 images belonging to 4 classes.
Found 120 images belonging to 4 classes.


In [3]:
train_ds.class_indices

{'comma': 0, 'exclamation': 1, 'period': 2, 'question': 3}

#### MODEL BUILDING & TRAINING

In [4]:
model = Sequential([
                    Conv2D(32, (3,3), input_shape=train_ds[0][0].shape[1:], activation="relu"),
                    #MaxPooling2D((2,2)),
                    Conv2D(64, (3,3), activation="relu"),
                    MaxPooling2D((2,2)),
                    Flatten(),
                    Dense(128, activation="relu"),
                    Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.L1L2()),
                    Dropout(0.3),
                    Dense(4, activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])
model.summary()   

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 128)               1179776   
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                        

In [5]:
model.fit(train_ds, epochs=30, validation_data=(val_ds))

Epoch 1/30
16/16 [==============================] - 4s 122ms/step - loss: 1.2149 - acc: 0.5635 - val_loss: 0.7154 - val_acc: 0.7750
Epoch 2/30
16/16 [==============================] - 0s 10ms/step - loss: 0.3871 - acc: 0.8689 - val_loss: 0.4267 - val_acc: 0.8167
Epoch 3/30
16/16 [==============================] - 0s 10ms/step - loss: 0.2182 - acc: 0.9180 - val_loss: 0.2469 - val_acc: 0.8917
Epoch 4/30
16/16 [==============================] - 0s 10ms/step - loss: 0.1483 - acc: 0.9385 - val_loss: 0.2462 - val_acc: 0.8500
Epoch 5/30
16/16 [==============================] - 0s 10ms/step - loss: 0.1380 - acc: 0.9508 - val_loss: 0.2108 - val_acc: 0.9167
Epoch 6/30
16/16 [==============================] - 0s 10ms/step - loss: 0.0877 - acc: 0.9570 - val_loss: 0.2472 - val_acc: 0.8833
Epoch 7/30
16/16 [==============================] - 0s 10ms/step - loss: 0.0645 - acc: 0.9795 - val_loss: 0.1765 - val_acc: 0.9083
Epoch 8/30
16/16 [==============================] - 0s 10ms/step - loss: 0.0466 - 

In [6]:
model.save('model\symbols.h5')